In [1]:
# load file
with open('data/republic.txt', 'r') as f:
    text = f.read()

text[:100]

'BOOK I.\n\nI went down yesterday to the Piraeus with Glaucon the son of Ariston,\nthat I might offer up'

In [2]:
import string
def clean(text):
    text = text.replace('--', ' ')
    tokens = text.split(' ')
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    tokens = [w for w in tokens if w.isalpha()]
    tokens = [w.lower() for w in tokens]
    return tokens
tokens = clean(text)
print(f'number of tokens: {len(tokens)}')
print(f'unique tokens: {len(set(tokens))}')

number of tokens: 96479
unique tokens: 6577


In [3]:
tokens[:10]

['book',
 'went',
 'down',
 'yesterday',
 'to',
 'the',
 'piraeus',
 'with',
 'glaucon',
 'the']

In [4]:
length = 51
sequences = []
for i in range(length, len(tokens)):
    seq = tokens[i - length: i]
    line = ' '.join(seq)
    sequences.append(seq)
print(f'Total # of lines: {len(sequences)}')

Total # of lines: 96428


In [5]:
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(sequences)
seqs = tokenizer.texts_to_sequences(sequences)

In [6]:
vocab_size = len(tokenizer.word_index) + 1

In [7]:
import numpy as np
from tensorflow.keras.utils import to_categorical
seqs = np.array(seqs)
X = seqs[:,:-1]
y = seqs[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_len = X.shape[1]

In [8]:
# build model
from tensorflow.keras.layers import LSTM, Embedding, Bidirectional, Input, Dense
from tensorflow.keras import Model, Sequential

inputs = Input(shape=(seq_len,))
embedding = Embedding(vocab_size, 50, input_length=seq_len)(inputs)
x = Bidirectional(LSTM(100, return_sequences=True))(embedding)
x = Bidirectional(LSTM(100, return_sequences=True))(x)
x = Bidirectional(LSTM(100))(x)
x = Dense(128, activation='relu')(x)
outputs = Dense(vocab_size, activation='softmax')(x)
model = Model(inputs=inputs, outputs=outputs, name='LSTM')

In [9]:
X.shape, y.shape

((96428, 50), (96428, 6578))

In [10]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [11]:
x_cut = X[:50000]
y_cut = y[:50000]
x_cut.shape, y_cut.shape

((50000, 50), (50000, 6578))

In [13]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_cut, y_cut, batch_size=32, epochs=50)

Epoch 1/50
1563/1563 [==============================] - 20s 10ms/step - loss: 6.1089 - accuracy: 0.0648
Epoch 2/50
1563/1563 [==============================] - 15s 10ms/step - loss: 5.7127 - accuracy: 0.0901
Epoch 3/50
1563/1563 [==============================] - 15s 10ms/step - loss: 5.5424 - accuracy: 0.1055
Epoch 4/50
1563/1563 [==============================] - 15s 10ms/step - loss: 5.3844 - accuracy: 0.1178
Epoch 5/50
1563/1563 [==============================] - 15s 10ms/step - loss: 5.2440 - accuracy: 0.1309
Epoch 6/50
1563/1563 [==============================] - 15s 10ms/step - loss: 5.1823 - accuracy: 0.1369
Epoch 7/50
1563/1563 [==============================] - 15s 10ms/step - loss: 5.0928 - accuracy: 0.1438
Epoch 8/50
1563/1563 [==============================] - 15s 10ms/step - loss: 5.0082 - accuracy: 0.1490
Epoch 9/50
1563/1563 [==============================] - 15s 10ms/step - loss: 4.9280 - accuracy: 0.1523
Epoch 10/50
1563/1563 [==============================] - 15s 10m

In [54]:
seed_text = sequences[np.random.randint(0, len(sequences))]
print(seed_text)

['he', 'replied', 'and', 'i', 'see', 'as', 'every', 'one', 'must', 'that', 'is', 'the', 'wretchedest', 'form', 'of', 'government', 'and', 'the', 'rule', 'of', 'a', 'in', 'estimating', 'the', 'men', 'too', 'may', 'i', 'not', 'fairly', 'make', 'a', 'like', 'i', 'should', 'have', 'a', 'judge', 'whose', 'mind', 'can', 'enter', 'into', 'and', 'see', 'nature', 'he', 'must', 'not', 'be', 'like']


In [39]:
encoded = tokenizer.texts_to_sequences(seed_text)

In [60]:
out = model.predict(encoded)

(51, 6578)

In [63]:
ind = np.argmax(out[-1])
for val, i in tokenizer.word_index.items():
    if i == ind:
        print(val)
        break

and


In [56]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
def generate(model, tokenizer, seq_length, seed_text, n_words):
    result = []
    in_text = seed_text
    for i in range(n_words):
        encoded = tokenizer.texts_to_sequences(in_text)
        encoded = pad_sequences(encoded, maxlen=seq_length, truncating='pre')
        out = model.predict(encoded)
        out = np.argmax(out[-1])
        out_word = ''
        for val, i in tokenizer.word_index.items():
            if i == out:
                out_word = val
                break
        print(out_word)
        in_text += ' ' + out_word
        result.append(out_word)
    return ' '.join(result)


In [57]:
res = generate(model, tokenizer, seq_len, seed_text, 50)

these
these
these
these
these
these
these
these
these
these
these
these
these
these
these
these
these
these
these
these
these
these
these
these
these
these
these
these
these
these
these
these
these
these
these
these
these
these
these
these
these
these
these
these
these
these
these
these
these
these


In [59]:
print(seed_text)
res

['he', 'replied', 'and', 'i', 'see', 'as', 'every', 'one', 'must', 'that', 'is', 'the', 'wretchedest', 'form', 'of', 'government', 'and', 'the', 'rule', 'of', 'a', 'in', 'estimating', 'the', 'men', 'too', 'may', 'i', 'not', 'fairly', 'make', 'a', 'like', 'i', 'should', 'have', 'a', 'judge', 'whose', 'mind', 'can', 'enter', 'into', 'and', 'see', 'nature', 'he', 'must', 'not', 'be', 'like', ' ', 't', 'h', 'e', 's', 'e', ' ', 't', 'h', 'e', 's', 'e', ' ', 't', 'h', 'e', 's', 'e', ' ', 't', 'h', 'e', 's', 'e', ' ', 't', 'h', 'e', 's', 'e', ' ', 't', 'h', 'e', 's', 'e', ' ', 't', 'h', 'e', 's', 'e', ' ', 't', 'h', 'e', 's', 'e', ' ', 't', 'h', 'e', 's', 'e', ' ', 't', 'h', 'e', 's', 'e', ' ', 't', 'h', 'e', 's', 'e', ' ', 't', 'h', 'e', 's', 'e', ' ', 't', 'h', 'e', 's', 'e', ' ', 't', 'h', 'e', 's', 'e', ' ', 't', 'h', 'e', 's', 'e', ' ', 't', 'h', 'e', 's', 'e', ' ', 't', 'h', 'e', 's', 'e', ' ', 't', 'h', 'e', 's', 'e', ' ', 't', 'h', 'e', 's', 'e', ' ', 't', 'h', 'e', 's', 'e', ' ', 't'

'these these these these these these these these these these these these these these these these these these these these these these these these these these these these these these these these these these these these these these these these these these these these these these these these these these'